In [1]:
!pip3 install pyro-ppl
!pip3 install numpy
!pip3 install arviz
!pip3 install matplotlib
!pip3 install torch
!pip3 install torchvision
!pip3 install arviz


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

import pyro
import arviz
from tqdm import tqdm

c:\Users\ohe.fi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Train the model on the MNIST dataset, using a 2-dimensional latent space. Plot this latent space in the same way as you did for PPCA, using colors to represent the different digits. Does it separate the classes better than PPCA?

In [3]:
cuda2 = False # torch.cuda.is_available() can be used to check if a gpu is available - I just set it to False
batch_size2 = 128
log_interval2 = 10
epochs2 = 10 # 10

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda2 else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda2 else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=False, **kwargs)

In [4]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc1a = nn.Linear(400, 100)
        self.fc21 = nn.Linear(100, 5) # Latent space of 2D
        self.fc22 = nn.Linear(100, 5) # Latent space of 2D
        self.fc3 = nn.Linear(5, 100) # Latent space of 2D
        self.fc3a = nn.Linear(100, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc1a(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.fc3a(h3))
        return torch.sigmoid(self.fc4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO


def train(epoch):
    model.train() # so that everything has gradients and we can do backprop and so on...
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward() # calc gradients
        train_loss += loss.item()
        optimizer.step() # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad(): # no_grad turns of gradients...
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(1, epochs2 + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 5).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'vae_sample_' + str(epoch) + '.png')

torch.save(model, 'vae2.pt')

In [10]:
model = torch.load('vae.pt')

In [20]:
def log_px_z(x, z):
    x_recon = model.decode(z).view(-1, 1, 28, 28)
    return -F.binary_cross_entropy(x_recon, x.view(-1, 1, 28, 28), reduction='sum')

def log_pt(x, parms, beta):
    log_like = log_px_z(x, parms['z'])
    log_prior = torch.distributions.MultivariateNormal(torch.zeros(5), torch.eye(5)).log_prob(parms['z'])

    return beta * log_like + log_prior

def log_pt_encoder(x, parms, beta):
    log_like = log_px_z(x, parms['z'])
    log_prior = torch.distributions.MultivariateNormal(torch.zeros(5), torch.eye(5)).log_prob(parms['z'])
    mu, logvar = model.encode(x.view(-1, 28*28))
    log_enc = torch.distributions.MultivariateNormal(mu, torch.eye(5)*torch.exp(0.5*logvar)).log_prob(parms['z'])
    return beta * (log_like + log_prior ) + (1 - beta) * log_enc

def log_IS_frac(x, z, m, s):
    q_z_x = torch.distributions.MultivariateNormal(m, torch.eye(5)*torch.exp(0.5*s)).log_prob(z)
    return log_pt(x, {'z': z}, 1) - q_z_x
    

In [21]:
def AIS_encoder(x, N, T):
    betas = np.linspace(0, 1, T)
    
    beta_diff = 1.0 / T
    chain_log_like = torch.zeros(N, dtype=torch.float64)
    mu, logvar = model.encode(x.view(-1, 28*28))
    for t in range(T-1):
        nuts = pyro.infer.NUTS(potential_fn=lambda parms: -log_pt_encoder(x, parms, betas[t]))
        
        mcmc = pyro.infer.MCMC(nuts, num_samples=N, warmup_steps=10, initial_params={'z': torch.randn(1, 5)}, disable_progbar=True)
        mcmc.run()

        z = mcmc.get_samples()['z']
        for n in range(N):
            log_prior = torch.distributions.MultivariateNormal(torch.zeros(5), torch.eye(5)).log_prob(z[n])[0]
            log_enc = torch.distributions.MultivariateNormal(mu, torch.eye(5)*torch.exp(0.5*logvar)).log_prob(z[n])[0]
            chain_log_like[n] += beta_diff * (log_px_z(x, z[n]) + log_prior - log_enc)

    if N == 1:
        return chain_log_like[0]
    
    chain_like = torch.exp(chain_log_like)
    return torch.log(torch.mean(chain_like))

def IS_encoder(x, N):
    log_fracs = torch.zeros(N, dtype=torch.float64)
    m, s = model.encode(x.view(-1, 28*28))
    for n in range(N):
        z = model.reparameterize(m, s)
        log_fracs[n] = log_IS_frac(x, z, m, s)
    
    return torch.log(torch.exp(log_fracs).mean())


In [ ]:
data = list(test_loader)
n_test_batch = 50

nlls = np.zeros(batch_size2 * n_test_batch)
for i in range(n_test_batch):
    x = data[i+1][0]
    for j in tqdm(range(batch_size2)):
        nlls[i*batch_size2+j] = -IS_encoder(x[j], 1000)
    print(np.mean(nlls[:batch_size2*(i+1)]))

In [ ]:
data = list(test_loader)
n_test_batch = 4

nlls = np.zeros(batch_size2 * n_test_batch)
for i in range(n_test_batch):
    x = data[i+1][0]
    for j in tqdm(range(batch_size2)):
        nlls[i*batch_size2+j] = -AIS_encoder(x[j], 1, 100)
    print(np.mean(nlls[:batch_size2*(i+1)]))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2080f7a-92d1-4485-9403-98db2db3ab51' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>